In [1]:
#
#

from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Import MNIST data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../mnist', one_hot=True)

from keras import backend as K
#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ../mnist\train-images-idx3-ubyte.gz
Extracting ../mnist\train-labels-idx1-ubyte.gz
Extracting ../mnist\t10k-images-idx3-ubyte.gz
Extracting ../mnist\t10k-labels-idx1-ubyte.gz


Using TensorFlow backend.


In [2]:
#Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_epoch = 5
logs_path = 'log/tensorboard_logs/'

#tf Graph input
x = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='InputData')
y = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='LabelData')

#Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

In [4]:
#Construct model and encapsulating all ops into scopes,
#Making tensorboard's graph visualization more convenient

with tf.name_scope('Model'):
    #Model
    y_pred = tf.nn.softmax(tf.matmul(x, W) + b)
with tf.name_scope('Loss'):
    #Minimize error using cross entropy
    loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_pred), reduction_indices=1))
with tf.name_scope('SGD'):
    #Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
with tf.name_scope('Accuracy'):
    #Accuracy
    acc = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
    acc = tf.reduce_mean(tf.cast(acc, dtype=tf.float32))
    
#Init global variables
init = tf.global_variables_initializer()

#Create a summary to monitor loss tensor
tf.summary.scalar('loss', loss)
#Create a summary to monitor accuracy tensor
tf.summary.scalar('accuracy', acc)
#Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [6]:
#Training
with tf.Session() as sess:
    sess.run(init)
    
    #op to write logs to tensorboard
    summary_writer = tf.summary.FileWriter(logdir=logs_path, graph=tf.get_default_graph())
    
    #Training
    for epoch in range(training_epochs):
        avg_loss = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        #Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            #Run optimization, loss op and summary nodes
            _, l, summary = sess.run([optimizer, loss, merged_summary_op], feed_dict={x: batch_x, y: batch_y})
            
            #Write logs at every iteration
            summary_writer.add_summary(summary, epoch*total_batch+i)
            
            #Compute average loss
            avg_loss = l / total_batch
            
        #Display logs per epoch step
        if(epoch+1) % display_epoch == 0:
            print('Epoch: %04d, loss= %.9f' % (epoch+1, avg_loss))
    
    print('Training Completed!!!')
    
    #Test model
    #Calculate Accuracy
    print('Accuracy: ', acc.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))
    
    print('Run the command line to show tensorboard: \n --> tensorboard --logdir=log/tensorboard_logs/ \n Then open http://0.0.0.0:6006/')

Epoch: 0005, loss= 0.000791848
Epoch: 0010, loss= 0.000685182
Epoch: 0015, loss= 0.000466076
Epoch: 0020, loss= 0.000417387
Epoch: 0025, loss= 0.000392401
Training Completed!!!
Accuracy:  0.9134
Run the command line to show tensorboard: 
 --> tensorboard -->logdir=log/tensorboard_logs/ 
 Then open http://0.0.0.0:6006/
